# Congress Trades Backtest (FMP + yfinance)

This notebook pulls **House + Senate** trading disclosures from\n
[Financial Modeling Prep](https://financialmodelingprep.com/) (FMP),\n
enriches them with market prices from **yfinance**, and runs a set of\n
backtests comparing:

- Entry on **transaction date** vs **filing date**\n
- **Long-only**, **short-only**, and **long+short** strategies\n
- **Equal-weighted** vs **mid-point-weighted** trades\n

We compute per-trade PnL and aggregate performance metrics per\n
member and per strategy (Sharpe, CAGR, drawdowns, Calmar, etc.).\n

> Requirements:
> - FMP account + API key in `.env` as `FMP_API_KEY`\n
> - Python env with `requests`, `pandas`, `numpy`, `yfinance`,\n
>   `python-dotenv`, `plotly` (already in `insiderscraper/requirements.txt`)


In [30]:
# --- Setup & Imports ---
from __future__ import annotations

import os
import sys
import datetime as dt
from dataclasses import dataclass
from typing import Literal, Iterable

import numpy as np
import pandas as pd
import requests
import yfinance as yf
import plotly.express as px
from dotenv import load_dotenv

# Locate project root (folder that contains insiderscraper/)
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Load .env from project root
load_dotenv(os.path.join(project_root, '.env'))

FMP_API_KEY = os.getenv('FMP_API_KEY')
if not FMP_API_KEY:
    raise RuntimeError("FMP_API_KEY not set in .env; please add it before running this notebook.")

# ---- Backtest configuration ----
BACKTEST_YEARS = 3  # how many years of data to request from FMP
TODAY = dt.date.today()
BACKTEST_START = TODAY - dt.timedelta(days=365 * BACKTEST_YEARS)
BACKTEST_END = TODAY

# Optional on-disk cache for raw FMP data (to avoid re-hitting the API)
DATA_DIR = os.path.join(project_root, 'insiderscraper', 'data')
os.makedirs(DATA_DIR, exist_ok=True)

print('Project root:', project_root)
print('Using FMP_API_KEY (set):', bool(FMP_API_KEY))
print('Backtest window:', BACKTEST_START, '->', BACKTEST_END)


Project root: /Users/justiny/Desktop/01 Projects/ 03 NUSSIF App/insiderscraper
Using FMP_API_KEY (set): True
Backtest window: 2023-02-23 -> 2026-02-22


In [31]:
# --- FMP client & congress trades fetch ---

FMP_BASE_URL = 'https://financialmodelingprep.com'
# NOTE: Double-check this endpoint name in FMP docs; adjust if needed.
# Some accounts use /api/v4/congress-trading or similar variants.
FMP_SENATE_PATH = '/stable/senate-latest'
FMP_HOUSE_PATH = '/stable/house-latest'

def fmp_get(path: str, params: dict | None = None) -> list[dict]:
    """GET helper that attaches the FMP API key and parses JSON.

    Raises RuntimeError on non-2xx responses. Returns list/dict JSON.
    """
    if params is None:
        params = {}
    params = {**params, 'apikey': FMP_API_KEY}
    url = FMP_BASE_URL + path
    resp = requests.get(url, params=params, timeout=30)
    if not resp.ok:
        raise RuntimeError(f'FMP request failed: {resp.status_code} {resp.text[:200]}')
    try:
        data = resp.json()
    except ValueError as exc:
        raise RuntimeError(f'Failed to decode JSON from FMP: {exc}') from exc
    return data

def parse_amount_range(amount_str: str | None) -> tuple[float | None, float | None, float | None]:
    """Parse textual amount ranges like '$1,001 - $15,000' into (min, max, mid).

    Returns (None, None, None) if parsing fails.
    """
    if not amount_str or not isinstance(amount_str, str):
        return None, None, None
    s = amount_str.replace('$', '').replace(',', '').strip()
    # Handle common patterns: '1001 - 15000' or '1001-15000' or '1001 to 15000'
    for sep in ['-', 'to', '–', '—']:
        if sep in s:
            parts = [p.strip() for p in s.split(sep)]
            if len(parts) == 2:
                try:
                    lo = float(parts[0])
                    hi = float(parts[1])
                    mid = 0.5 * (lo + hi)
                    return lo, hi, mid
                except ValueError:
                    return None, None, None
    # Fallback: try single number
    try:
        val = float(s)
        return val, val, val
    except ValueError:
        return None, None, None

def normalize_fmp_trades(raw: list[dict]) -> pd.DataFrame:
    """Normalize raw FMP congress trades JSON into a tidy DataFrame.
    
    Compatible with FMP 'Stable' endpoints (/stable/senate-latest, etc.)
    and previous v4 structures.
    """
    if not raw:
        return pd.DataFrame()
        
    df = pd.DataFrame(list(raw))

    # 1. Handle Names (New API uses firstName/lastName)
    if 'firstName' in df.columns and 'lastName' in df.columns:
        df['member_name'] = df['firstName'].fillna('') + " " + df['lastName'].fillna('')
    elif 'name' in df.columns:
        df['member_name'] = df['name']
    elif 'politician' in df.columns:
        df['member_name'] = df['politician']
    else:
        df['member_name'] = 'Unknown'

    # 2. Map raw columns to canonical names
    # Key: FMP raw field, Value: Our target column name
    col_map = {
        'symbol': 'ticker',
        'transactionDate': 'transaction_date',
        'disclosureDate': 'filing_date',
        'type': 'transaction_type_raw',
        'amount': 'amount_range_raw',
        'assetDescription': 'asset_name',
        'office': 'chamber',
        'owner': 'owner'
    }

    # Identify which columns actually exist in this specific payload
    existing_cols = {raw_key: target_key for raw_key, target_key in col_map.items() if raw_key in df.columns}
    
    # Create the output dataframe with renamed columns
    out = df.rename(columns=existing_cols)
    
    # Keep the member_name we generated in step 1
    out['member_name'] = df['member_name']

    # 3. Normalise dates to datetime.date objects
    for dcol in ['transaction_date', 'filing_date']:
        if dcol in out.columns:
            out[dcol] = pd.to_datetime(out[dcol], errors='coerce').dt.date

    # 4. Normalise transaction type to BUY/SELL/OTHER
    def _norm_tx_type(x: str | None) -> str:
        if not isinstance(x, str):
            return 'OTHER'
        s = x.strip().upper()
        if any(w in s for w in ['BUY', 'PURCHASE']):
            return 'BUY'
        if any(w in s for w in ['SELL', 'SALE']):
            return 'SELL'
        return 'OTHER'

    if 'transaction_type_raw' in out.columns:
        out['transaction_type'] = out['transaction_type_raw'].map(_norm_tx_type)
    else:
        out['transaction_type'] = 'OTHER'

    # 5. Parse amount range into numeric values
    if 'amount_range_raw' in out.columns:
        mins, maxs, mids = [], [], []
        for v in out['amount_range_raw']:
            lo, hi, mid = parse_amount_range(v)
            mins.append(lo)
            maxs.append(hi)
            mids.append(mid)
        out['amount_min'] = mins
        out['amount_max'] = maxs
        out['mid_point'] = mids
    else:
        out['amount_min'] = np.nan
        out['amount_max'] = np.nan
        out['mid_point'] = np.nan

    # 6. Final Clean up: Keep only relevant columns
    cols_to_keep = [
        'member_name', 'ticker', 'asset_name', 'chamber', 
        'transaction_date', 'filing_date', 'transaction_type', 
        'amount_min', 'amount_max', 'mid_point', 'owner'
    ]
    # Only keep columns that were actually successfully created
    final_cols = [c for c in cols_to_keep if c in out.columns]
    
    return out[final_cols].copy()

def fetch_congress_trades(
    start_date: dt.date,
    end_date: dt.date,
    chamber_filter: str | None = None,
) -> pd.DataFrame:
    """Fetch recent trades and filter by date locally."""
    
    all_raw_data = []
    
    # Decide which endpoints to hit
    endpoints = []
    if chamber_filter is None:
        endpoints = [FMP_SENATE_PATH, FMP_HOUSE_PATH]
    elif 'senate' in chamber_filter.lower():
        endpoints = [FMP_SENATE_PATH]
    else:
        endpoints = [FMP_HOUSE_PATH]

    for path in endpoints:
        # The new API uses 'page' instead of date filters
        # We loop through pages until we find trades older than our start_date
        for page in range(1):  # Adjust page limit as needed
            params = {'page': page, 'limit': 25}
            batch = fmp_get(path, params=params)
            
            if not batch:
                break
                
            all_raw_data.extend(batch)
            
            # Optimization: check if the last item in batch is already older than start_date
            last_date_str = batch[-1].get('transactionDate')
            if last_date_str:
                last_date = pd.to_datetime(last_date_str).date()
                if last_date < start_date:
                    break
    
    df = normalize_fmp_trades(all_raw_data)
    
    # Filter by date range locally
    if not df.empty:
        mask = (df['transaction_date'] >= start_date) & (df['transaction_date'] <= end_date)
        df = df[mask].copy()
        
    return df

def load_or_fetch_congress_trades(
    start_date: dt.date,
    end_date: dt.date,
    chamber_filter: str | None = None,
) -> pd.DataFrame:
    """Load trades from a local cache if available, otherwise call FMP.
    """
    tag = f'{start_date}_{end_date}_{chamber_filter or "all"}'.replace(' ', '')
    cache_path = os.path.join(DATA_DIR, f'fmp_congress_trades_{tag}.parquet')
    if os.path.exists(cache_path):
        print('Loading cached FMP trades from', cache_path)
        return pd.read_parquet(cache_path)

    print('Fetching FMP congress trades from API...')
    df = fetch_congress_trades(start_date, end_date, chamber_filter=chamber_filter)
    if not df.empty:
        df.to_parquet(cache_path, index=False)
        print('Saved cache to', cache_path)
    else:
        print('No FMP trades returned for this window.')
    return df

# Quick sanity check (small window to avoid heavy API usage on first run)
sample_start = BACKTEST_START
sample_end = BACKTEST_START + dt.timedelta(days=365*3)
fmp_sample = load_or_fetch_congress_trades(sample_start, sample_end, chamber_filter=None)
print('Sample FMP trades shape:', fmp_sample.shape)
fmp_sample.head()


Loading cached FMP trades from /Users/justiny/Desktop/01 Projects/ 03 NUSSIF App/insiderscraper/insiderscraper/data/fmp_congress_trades_2023-02-23_2026-02-22_all.parquet
Sample FMP trades shape: (50, 11)


,member_name,ticker,asset_name,chamber,transaction_date,filing_date,transaction_type,amount_min,amount_max,mid_point,owner
0,John Boozman,MU,Micron Technology Inc,John Boozman,2026-01-08,2026-02-15,SELL,1001.0,15000.0,8000.5,Joint
1,John Boozman,UNP,Union Pacific Corp,John Boozman,2026-01-29,2026-02-15,SELL,1001.0,15000.0,8000.5,Joint
2,John Boozman,RNWGX,NEW WORLD FUND INC,John Boozman,2026-01-23,2026-02-15,BUY,1001.0,15000.0,8000.5,Joint
3,John Boozman,XOM,Exxon Mobil Corp,John Boozman,2026-01-08,2026-02-15,BUY,1001.0,15000.0,8000.5,Joint
4,John Boozman,SLV,iShares Silver Trust,John Boozman,2026-01-23,2026-02-15,SELL,1001.0,15000.0,8000.5,Joint


In [32]:
# --- Price data via yfinance (in-memory, no DB dependency) ---

def download_price_history(
    tickers: list[str],
    start_date: dt.date,
    end_date: dt.date,
    *,
    buffer_days: int = 10,
) -> pd.DataFrame:
    """Download daily adjusted close prices for tickers into long-form DF.

    Returns DataFrame with columns [date, ticker, close].
    """
    if not tickers:
        return pd.DataFrame(columns=['date', 'ticker', 'close'])

    uniq = sorted({t for t in tickers if isinstance(t, str) and t.strip()})
    if not uniq:
        return pd.DataFrame(columns=['date', 'ticker', 'close'])

    start = start_date - dt.timedelta(days=buffer_days)
    end = end_date + dt.timedelta(days=buffer_days)
    print(f'Downloading price history for {len(uniq)} tickers from', start, 'to', end)

    data = yf.download(uniq, start=start, end=end, progress=False, auto_adjust=True)
    if data.empty:
        return pd.DataFrame(columns=['date', 'ticker', 'close'])

    # yfinance returns either a Series, DataFrame with columns, or multiindex columns.
    if isinstance(data, pd.Series):
        # Single ticker, Series indexed by date
        df = data.to_frame(name='close').reset_index().rename(columns={'index': 'date'})
        df['ticker'] = uniq[0]
    else:
        # Prefer 'Adj Close' if present, else 'Close'
        if isinstance(data.columns, pd.MultiIndex):
            if ('Adj Close' in data.columns.get_level_values(0)):
                px = data['Adj Close']
            else:
                px = data['Close']
            df = px.reset_index().melt(id_vars=['Date'], var_name='ticker', value_name='close')
            df = df.rename(columns={'Date': 'date'})
        else:
            # Single ticker, DataFrame with 'Adj Close' or 'Close'
            col = 'Adj Close' if 'Adj Close' in data.columns else 'Close'
            df = data[[col]].reset_index().rename(columns={'Date': 'date', col: 'close'})
            df['ticker'] = uniq[0]

    df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.date
    df = df.dropna(subset=['date', 'close'])
    return df[['date', 'ticker', 'close']]

def get_price_on_or_before_from_df(
    prices: pd.DataFrame,
    ticker: str,
    target_date: dt.date,
) -> float | None:
    """Return last available close for ticker on or before target_date.
    """
    mask = (prices['ticker'] == ticker) & (prices['date'] <= target_date)
    sub = prices.loc[mask]
    if sub.empty:
        return None
    row = sub.sort_values('date').iloc[-1]
    return float(row['close'])

# Example: price snapshot for sample FMP trades (if any)
if not fmp_sample.empty:
    sample_tickers = fmp_sample['ticker'].dropna().astype(str).unique().tolist() if 'ticker' in fmp_sample.columns else []
    price_sample = download_price_history(sample_tickers, sample_start, sample_end)
    print('Sample price DF shape:', price_sample.shape)
    price_sample.head()
else:
    price_sample = pd.DataFrame(columns=['date', 'ticker', 'close'])
    print('No sample FMP trades to fetch prices for.')


Sample price DF shape: (35736, 3)


In [33]:
# --- Strategy engine (entry/exit logic, sizing, direction) ---

EntryMode = Literal['TX', 'FILING']
HoldMode = Literal['TODAY', 'N_DAYS']
SizeMode = Literal['MIDPOINT', 'EQUAL']
DirectionMode = Literal['LONG_ONLY', 'SHORT_ONLY', 'LONG_SHORT']

@dataclass
class StrategyConfig:
    entry_mode: EntryMode
    hold_mode: HoldMode
    hold_days: int | None
    size_mode: SizeMode
    direction_mode: DirectionMode

    def key(self) -> str:
        return f'{self.entry_mode}_{self.hold_mode}_{self.hold_days or 0}_{self.size_mode}_{self.direction_mode}'

def apply_strategy_fmp(
    trades: pd.DataFrame,
    prices: pd.DataFrame,
    cfg: StrategyConfig,
) -> pd.DataFrame:
    """Attach per-trade PnL columns for a given strategy configuration.

    - Chooses entry_date from transaction_date vs filing_date.
    - Computes exit_date based on hold_mode/hold_days.
    - Looks up entry_price/exit_price from the prices DF.
    - Sets direction based on transaction_type and direction_mode.
    - Applies size_mode (midpoint vs equal).
    """
    if trades.empty:
        return trades.assign(
            strategy_key=[],
            entry_date=[],
            exit_date=[],
            entry_price=[],
            exit_price=[],
            direction=[],
            notional=[],
            trade_return=[],
            pnl_dollars=[],
        )

    df = trades.copy()

    # --- Entry date ---
    if cfg.entry_mode == 'TX':
        entry_date = df['transaction_date']
    elif cfg.entry_mode == 'FILING':
        entry_date = df['filing_date']
    else:
        raise ValueError(f'Unknown entry_mode: {cfg.entry_mode}')

    # --- Direction: BUY=+1, SELL=-1, other=0 ---
    tx_type = df.get('transaction_type', pd.Series(index=df.index, dtype=str)).fillna('OTHER')
    base_dir = np.where(
        tx_type == 'BUY',
        1.0,
        np.where(tx_type == 'SELL', -1.0, 0.0),
    )
    if cfg.direction_mode == 'LONG_ONLY':
        direction = np.where(base_dir > 0, 1.0, 0.0)
    elif cfg.direction_mode == 'SHORT_ONLY':
        direction = np.where(base_dir < 0, -1.0, 0.0)
    elif cfg.direction_mode == 'LONG_SHORT':
        direction = base_dir
    else:
        raise ValueError(f'Unknown direction_mode: {cfg.direction_mode}')

    # --- Sizing ---
    if cfg.size_mode == 'MIDPOINT':
        notional = df.get('mid_point', pd.Series(index=df.index, dtype=float)).fillna(0.0)
    elif cfg.size_mode == 'EQUAL':
        notional = np.where(direction != 0.0, 1.0, 0.0)
    else:
        raise ValueError(f'Unknown size_mode: {cfg.size_mode}')

    # --- Entry & exit prices via in-memory price DF ---
    # Convert entry_date to pure date objects (handles pandas Timestamp)
    entry_date_clean = pd.to_datetime(entry_date, errors='coerce').dt.date

    # Entry price: price on or before entry_date
    entry_price_vals = []
    for t, d in zip(df.get('ticker'), entry_date_clean):
        if not isinstance(t, str) or d is None:
            entry_price_vals.append(np.nan)
        else:
            p = get_price_on_or_before_from_df(prices, t, d)
            entry_price_vals.append(np.nan if p is None else p)

    entry_price = pd.to_numeric(entry_price_vals, errors='coerce')

    # Exit date & price
    if cfg.hold_mode == 'TODAY':
        exit_date = pd.Series(TODAY, index=df.index)
    elif cfg.hold_mode == 'N_DAYS':
        if cfg.hold_days is None or cfg.hold_days <= 0:
            raise ValueError('hold_days must be positive for N_DAYS mode')
        exit_date = entry_date_clean + pd.to_timedelta(cfg.hold_days, unit='D')
    else:
        raise ValueError(f'Unknown hold_mode: {cfg.hold_mode}')

    exit_date_clean = pd.to_datetime(exit_date, errors='coerce').dt.date
    exit_price_vals = []
    for t, d in zip(df.get('ticker'), exit_date_clean):
        if not isinstance(t, str) or d is None:
            exit_price_vals.append(np.nan)
        else:
            p = get_price_on_or_before_from_df(prices, t, d)
            exit_price_vals.append(np.nan if p is None else p)

    exit_price = pd.to_numeric(exit_price_vals, errors='coerce')

    # --- Returns & PnL ---
    valid_mask = (entry_price > 0) & (exit_price > 0) & (direction != 0.0)
    raw_ret = np.where(
        valid_mask,
        (exit_price - entry_price) / entry_price,
        np.nan,
    )
    trade_return = direction * raw_ret
    pnl_dollars = notional * trade_return

    out = df.assign(
        strategy_key=cfg.key(),
        entry_date=entry_date_clean,
        exit_date=exit_date_clean,
        entry_price=entry_price,
        exit_price=exit_price,
        direction=direction,
        notional=notional,
        trade_return=trade_return,
        pnl_dollars=pnl_dollars,
    )
    return out

def run_strategies(
    trades: pd.DataFrame,
    prices: pd.DataFrame,
    configs: list[StrategyConfig],
) -> pd.DataFrame:
    """Run multiple strategy configs and concatenate results.
    """
    results = []
    for cfg in configs:
        print('Running strategy:', cfg.key())
        res = apply_strategy_fmp(trades, prices, cfg)
        results.append(res)
    if not results:
        return pd.DataFrame()
    return pd.concat(results, ignore_index=True)


In [34]:
# --- Metrics & aggregation ---

def compute_sharpe(returns: pd.Series, periods_per_year: int = 252) -> float:
    r = returns.dropna()
    if len(r) < 2:
        return float('nan')
    mu = r.mean()
    sigma = r.std(ddof=1)
    if sigma == 0:
        return float('nan')
    return float(mu / sigma * np.sqrt(periods_per_year))

def compute_sortino(returns: pd.Series, periods_per_year: int = 252) -> float:
    r = returns.dropna()
    if len(r) < 2:
        return float('nan')
    downside = r[r < 0]
    if downside.empty:
        return float('nan')
    dd = downside.std(ddof=1)
    if dd == 0:
        return float('nan')
    mu = r.mean()
    return float(mu / dd * np.sqrt(periods_per_year))

def summarize_member_strategy(trades_with_pnl: pd.DataFrame) -> pd.DataFrame:
    """Aggregate per-trade results into per-member, per-strategy metrics.

    Expects columns: member_name, strategy_key, trade_return, notional,\n
    pnl_dollars, entry_date, exit_date.
    """
    if trades_with_pnl.empty:
        return pd.DataFrame()

    df = trades_with_pnl.dropna(subset=['trade_return']).copy()

    def _agg(group: pd.DataFrame) -> pd.Series:
        g = group.sort_values('exit_date')
        rets = g['trade_return']
        equity = (1.0 + rets).cumprod()
        start_date = g['entry_date'].min()
        end_date = g['exit_date'].max()
        if pd.notna(start_date) and pd.notna(end_date):
            days = max((end_date - start_date).days, 1)
        else:
            days = 1
        years = days / 365.25
        if years <= 0:
            cagr = float('nan')
        else:
            cagr = float(equity.iloc[-1] ** (1.0 / years) - 1.0)
        drawdown = equity / equity.cummax() - 1.0
        max_dd = float(drawdown.min())
        calmar = float('nan')
        if max_dd < 0 and not np.isnan(cagr):
            calmar = float(cagr / abs(max_dd))
        sharpe = compute_sharpe(rets)
        sortino = compute_sortino(rets)
        return pd.Series({
            'n_trades': len(g),
            'total_notional': g['notional'].sum(),
            'total_pnl': g['pnl_dollars'].sum(),
            'avg_return': rets.mean(),
            'median_return': rets.median(),
            'win_rate': (rets > 0).mean(),
            'sharpe': sharpe,
            'sortino': sortino,
            'cagr': cagr,
            'max_drawdown': max_dd,
            'calmar': calmar,
        })

    group_cols = []
    if 'member_name' in df.columns:
        group_cols.append('member_name')
    if 'chamber' in df.columns:
        group_cols.append('chamber')
    group_cols.append('strategy_key')

    summary = (
        df.groupby(group_cols, as_index=False)
          .apply(_agg)
          .reset_index(drop=True)
    )
    return summary


In [35]:
# --- Example workflow: run a few strategies and visualise rankings ---

# 1) Load full backtest window of FMP trades (from cache or API)
trades_all = load_or_fetch_congress_trades(BACKTEST_START, BACKTEST_END, chamber_filter=None)
print('All trades shape:', trades_all.shape)
trades_all.head()

# 2) Download prices for all tickers in this universe
if 'ticker' in trades_all.columns:
    all_tickers = trades_all['ticker'].dropna().astype(str).unique().tolist()
else:
    all_tickers = []

prices_all = download_price_history(all_tickers, BACKTEST_START, BACKTEST_END)
print('Prices DF shape:', prices_all.shape)

# 3) Configure strategies: TX vs FILING; MIDPOINT-sized; long-only; mark-to-market today
configs = [
    StrategyConfig('TX', 'TODAY', None, 'MIDPOINT', 'LONG_ONLY'),
    StrategyConfig('FILING', 'TODAY', None, 'MIDPOINT', 'LONG_ONLY'),
    # Example N_DAYS horizon: 20 trading days (approx)
    StrategyConfig('TX', 'N_DAYS', 20, 'MIDPOINT', 'LONG_ONLY'),
    StrategyConfig('FILING', 'N_DAYS', 20, 'MIDPOINT', 'LONG_ONLY'),
]

all_trades_pnl = run_strategies(trades_all, prices_all, configs)
print('All trades with PnL shape:', all_trades_pnl.shape)
all_trades_pnl.head()

# 4) Aggregate per member & strategy
summary = summarize_member_strategy(all_trades_pnl)
print('Summary shape:', summary.shape)
summary.head()

# 5) Visualise: top 20 members by Sharpe for a chosen strategy
strategy_to_plot = configs[0].key()  # e.g. TX entry, mark-to-market today
plot_df = (
    summary[summary['strategy_key'] == strategy_to_plot]
      .sort_values('sharpe', ascending=False)
      .head(20)
)

fig = px.bar(
    plot_df,
    x='member_name',
    y='sharpe',
    color='chamber' if 'chamber' in plot_df.columns else None,
    title=f'Top members by Sharpe ({strategy_to_plot})',
)
print(plot_df)
fig.show()


Loading cached FMP trades from /Users/justiny/Desktop/01 Projects/ 03 NUSSIF App/insiderscraper/insiderscraper/data/fmp_congress_trades_2023-02-23_2026-02-22_all.parquet
All trades shape: (50, 11)
Prices DF shape: (35736, 3)
Running strategy: TX_TODAY_0_MIDPOINT_LONG_ONLY
Running strategy: FILING_TODAY_0_MIDPOINT_LONG_ONLY
Running strategy: TX_N_DAYS_20_MIDPOINT_LONG_ONLY
Running strategy: FILING_N_DAYS_20_MIDPOINT_LONG_ONLY
All trades with PnL shape: (200, 20)
Summary shape: (20, 14)
         member_name           chamber                   strategy_key  \
3   Gilbert Cisneros  Gilbert Cisneros  TX_TODAY_0_MIDPOINT_LONG_ONLY   
7       John Boozman      John Boozman  TX_TODAY_0_MIDPOINT_LONG_ONLY   
11        Kevin Hern        Kevin Hern  TX_TODAY_0_MIDPOINT_LONG_ONLY   
15  Richard W. Allen  Richard W. Allen  TX_TODAY_0_MIDPOINT_LONG_ONLY   
19    Thomas H. Kean    Thomas H. Kean  TX_TODAY_0_MIDPOINT_LONG_ONLY   

    n_trades  total_notional    total_pnl  avg_return  median_return  \

/var/folders/yl/gn9fldpj69169vhn466y4klh0000gn/T/ipykernel_87685/1659799676.py:82: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

